# X2Paddle快速上手——TensorFlow迁移至PaddlePaddle
***X2Paddle简介***：X2Paddle支持将Caffe/TensorFlow/ONNX/PyTorch深度学习框架训练得到的模型，迁移至PaddlePaddle模型。   
***X2Paddle代码GitHub链接***：[https://github.com/PaddlePaddle/X2Paddle](https://github.com/PaddlePaddle/X2Paddle)  
***【注意】***前往GitHub给[X2Paddle](https://github.com/PaddlePaddle/X2Paddle)点击Star，关注项目，即可随时了解X2Paddle的最新进展。  
本教程用于帮助用户学习将TensorFlow训练后的预测模型迁移至PaddlePaddle框架，以TensorFlow版本的[MobileNetV1](https://github.com/tensorflow/models/tree/master/research/slim)为例进行详细介绍。  

## 安装及准备
### 1. 安装X2Paddle
***方式一：（推荐）***

In [ ]:
! git clone https://github.com/PaddlePaddle/X2Paddle.git
! cd X2Paddle
! git checkout develop
! python setup.py install

***方式二：***

In [ ]:
! pip install x2paddle==1.0.1 --index https://pypi.Python.org/simple/

### 2. 安装TensorFlow

In [ ]:
! pip install tensorflow==1.14.0

### 3. 安装PaddlePaddle

In [ ]:
! pip install paddlepaddle==2.0.1

## 模型迁移
### 1. 获取MobileNetV1的FrozenModel
由于X2Paddle只支持TensorFlow中FrozenModel的转换，如果为纯checkpoint模型，需要参考参考X2Paddle官方[文档](https://github.com/PaddlePaddle/X2Paddle/blob/develop/docs/user_guides/export_tf_model.md)，将其转换为FrozenModel，本示例中提供的模型为FrozenModel，所以无需转换。

In [ ]:
! wget http://download.tensorflow.org/models/mobilenet_v1_2018_02_22/mobilenet_v1_0.25_128.tgz
! tar zxvf mobilenet_v1_0.25_128.tgz

### 2. 转换
需要传入的参数如下：
> --framework (-f)：源模型类型，此处设置为tensorflow。  
> --save_dir (-s)：指定转换后的模型保存目录路径。  
> --model (-m)：指定tensorflow的pb模型。  
> --paddle_type (-pt)：指定转换为动态图代码(dygraph)或者静态图代码(static)，默认为dygraph。  

***方式一：***生成静态图代码，并保存成静态图预测模型

In [ ]:
! x2paddle -f tensorflow -m ./mobilenet_v1_0.25_128_frozen.pb -s pd_model_static -pt static

***方式二：***生成动态图代码，并保存成静态图预测模型


In [ ]:
! x2paddle -f tensorflow -m ./mobilenet_v1_0.25_128_frozen.pb  -s pd_model_dygraph -pt dygraph

## PaddlePaddle模型使用
使用方式一转换得到的PaddlePaddle预测模型进行预测：  
（1）下载ImageNet类别文件

In [ ]:
! wget https://raw.githubusercontent.com/Lasagne/Recipes/master/examples/resnet50/imagenet_classes.txt

（2）预测

In [ ]:
# 构造输入
import cv2
import numpy as np
img = cv2.imread("dog_tf.png").astype("float32") / 255.0
img = np.expand_dims(img, 0)
img -= 0.5
img *= 2.0
    
# 进行预测
import paddle
import numpy as np
paddle.enable_static()
exe = paddle.static.Executor(paddle.CPUPlace())
[prog, inputs, outputs] = paddle.static.load_inference_model(path_prefix="pd_model_static/inference_model", 
                                                             executor=exe, 
                                                             model_filename="model.pdmodel",
                                                             params_filename="model.pdiparams")
result = exe.run(prog, feed={inputs[0]: img}, fetch_list=outputs)
max_index = np.argmax(result[0])
with open('imagenet_classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]
print("The category of dog.jpg is: {}".format(classes[max_index]))